In [ ]:
import pandas as pd
import numpy as np
import gseapy as gp
import matplotlib.pyplot as plt
import seaborn as sns
from tkinter import W
from matplotlib import rcParams

In [ ]:
gene_df_H = pd.read_csv("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/ensemble_gene_attributions_Not_AD.csv")
top_genes_H = gene_df_H.sort_values('mean_attribution', ascending=False)['gene'].head(50).tolist()
enr_H=gp.enrichr(gene_list=top_genes_H,gene_sets=['GO_Biological_Process_2025','KEGG_2021_Human','Reactome_Pathways_2024'],organism='Human',outdir='/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot',cutoff=1)

gene_df_M = pd.read_csv("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/ensemble_gene_attributions_Low.csv")
top_genes_M = gene_df_M.sort_values('mean_attribution', ascending=False)['gene'].head(50).tolist()
enr_M=gp.enrichr(gene_list=top_genes_M,gene_sets=['GO_Biological_Process_2025','KEGG_2021_Human','Reactome_Pathways_2024'],organism='Human',outdir='/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot',cutoff=1)

gene_df_S = pd.read_csv("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/ensemble_gene_attributions_Intermediate.csv")
top_genes_S = gene_df_S.sort_values('mean_attribution', ascending=False)['gene'].head(50).tolist()
enr_S=gp.enrichr(gene_list=top_genes_S,gene_sets=['GO_Biological_Process_2025','KEGG_2021_Human','Reactome_Pathways_2024'],organism='Human',outdir='/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot',cutoff=1)

gene_df_W = pd.read_csv("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/ensemble_gene_attributions_High.csv")
top_genes_W = gene_df_W.sort_values('mean_attribution', ascending=False)['gene'].head(50).tolist()
enr_W=gp.enrichr(gene_list=top_genes_W,gene_sets=['GO_Biological_Process_2025','KEGG_2021_Human','Reactome_Pathways_2024'],organism='Human',outdir='/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot',cutoff=1)



In [ ]:
enr_H_results = enr_H.results.copy()
enr_H_results['Group'] = 'Normal'

enr_M_results = enr_M.results.copy()
enr_M_results['Group'] = 'Low'

enr_S_results = enr_S.results.copy()
enr_S_results['Group'] = 'Intermediate'

enr_W_results = enr_W.results.copy()
enr_W_results['Group'] = 'High'


In [ ]:
combined_results = pd.concat([enr_H_results, enr_M_results, enr_S_results, enr_W_results], ignore_index=True)
significant_results = combined_results[combined_results['Adjusted P-value'] < 0.05]

In [ ]:
rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
rcParams['axes.unicode_minus'] = False

def create_top_pathways_heatmap(enr_H, enr_M, enr_S, enr_W, gene_set_filter='GO_Biological_Process_2025', top_n=10, path=None):

    def get_top_pathways(enr_result, gene_set, n=top_n):
        df = enr_result.results.copy()
        df_filtered = df[(df['Gene_set'] == gene_set) & (df['Adjusted P-value'] < 0.05)]
        top_pathways = df_filtered.nsmallest(n, 'Adjusted P-value')
        return top_pathways[['Term', 'Adjusted P-value', 'Odds Ratio']]

    h_top = get_top_pathways(enr_H, gene_set_filter, top_n)
    m_top = get_top_pathways(enr_M, gene_set_filter, top_n)
    s_top = get_top_pathways(enr_S, gene_set_filter, top_n)
    w_top = get_top_pathways(enr_W, gene_set_filter, top_n)

    all_pathways = pd.concat([h_top['Term'], m_top['Term'], s_top['Term'], w_top['Term']]).unique()

    heatmap_data = pd.DataFrame(index=all_pathways, columns=['normal', 'Low', 'Intermediate','High'])

    for pathway in all_pathways:
        h_match = h_top[h_top['Term'] == pathway]
        if len(h_match) > 0:
            heatmap_data.loc[pathway, 'normal'] = -np.log10(h_match['Adjusted P-value'].iloc[0])
        else:
            h_all = enr_H.results[(enr_H.results['Gene_set'] == gene_set_filter) & 
                                 (enr_H.results['Term'] == pathway)]
            if len(h_all) > 0:
                heatmap_data.loc[pathway, 'normal'] = -np.log10(h_all['Adjusted P-value'].iloc[0])
            else:
                heatmap_data.loc[pathway, 'normal'] = 0

        m_match = m_top[m_top['Term'] == pathway]
        if len(m_match) > 0:
            heatmap_data.loc[pathway, 'Low'] = -np.log10(m_match['Adjusted P-value'].iloc[0])
        else:
            m_all = enr_M.results[(enr_M.results['Gene_set'] == gene_set_filter) & 
                                 (enr_M.results['Term'] == pathway)]
            if len(m_all) > 0:
                heatmap_data.loc[pathway, 'Low'] = -np.log10(m_all['Adjusted P-value'].iloc[0])
            else:
                heatmap_data.loc[pathway, 'Low'] = 0

        s_match = s_top[s_top['Term'] == pathway]
        if len(s_match) > 0:
            heatmap_data.loc[pathway, 'Intermediate'] = -np.log10(s_match['Adjusted P-value'].iloc[0])
        else:
            s_all = enr_S.results[(enr_S.results['Gene_set'] == gene_set_filter) & 
                                 (enr_S.results['Term'] == pathway)]
            if len(s_all) > 0:
                heatmap_data.loc[pathway, 'Intermediate'] = -np.log10(s_all['Adjusted P-value'].iloc[0])
            else:
                heatmap_data.loc[pathway, 'Intermediate'] = 0
    
    
        w_match = w_top[w_top['Term'] == pathway]
        if len(w_match) > 0:
            heatmap_data.loc[pathway, 'High'] = -np.log10(w_match['Adjusted P-value'].iloc[0])
        else:
            w_all = enr_W.results[(enr_W.results['Gene_set'] == gene_set_filter) & 
                                 (enr_W.results['Term'] == pathway)]
            if len(w_all) > 0:
                heatmap_data.loc[pathway, 'High'] = -np.log10(w_all['Adjusted P-value'].iloc[0])
            else:
                heatmap_data.loc[pathway, 'High'] = 0
    
    heatmap_data = heatmap_data.astype(float)

    plt.figure(figsize=(10, max(8, len(all_pathways) * 0.4)))
    
    ax = sns.heatmap(heatmap_data, 
                     annot=True,  
                     fmt='.2f',   
                     cmap='Reds', 
                     cbar_kws={'label': '-log10(Adjusted P-value)'},
                     linewidths=0.5,
                     linecolor='white')
    
    plt.title(f'Top {top_n} Pathways Heatmap\n{gene_set_filter}', fontsize=14, pad=20)
    plt.xlabel('Groups', fontsize=12)
    plt.ylabel('Pathways', fontsize=12)

    pathway_labels = [pathway[:60] + '...' if len(pathway) > 60 else pathway 
                     for pathway in heatmap_data.index]
    ax.set_yticklabels(pathway_labels, rotation=0, fontsize=9)
    
    ax.set_xticklabels(['normal', 'Low', 'Intermediate','High'], rotation=0, fontsize=11)

    significance_line = -np.log10(0.05)
    
    plt.tight_layout()
    plt.savefig(f"/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot/top_{top_n}_pathways_heatmap_{gene_set_filter}_50.pdf", dpi=300, bbox_inches='tight')
    plt.show()
    

    if path:
        plt.savefig(path, dpi=300)
    
    return heatmap_data, all_pathways

heatmap_reactome, pathways_reactome = create_top_pathways_heatmap(enr_H, enr_M, enr_S, enr_W,
                                                                  'Reactome_Pathways_2024', 
                                                                  top_n=5,
                                                                  path="/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD/Analysis/Figure/GSEA_plot/Pathways.pdf")